In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
!pip install stable-baselines3[extra]

In [5]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

# 2. Load Environment

In [6]:
environment_name = "CartPole-v0"

In [7]:
env = gym.make(environment_name)

In [8]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        print(f'n_state = {n_state}, reward = {reward}, done = {done}')
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

n_state = [-0.01992185 -0.20394045  0.0479903   0.34802204], reward = 1.0, done = False
n_state = [-0.02400066 -0.39971095  0.05495074  0.65544343], reward = 1.0, done = False
n_state = [-0.03199488 -0.20539537  0.06805961  0.3805574 ], reward = 1.0, done = False
n_state = [-0.03610279 -0.40141445  0.07567076  0.69389933], reward = 1.0, done = False
n_state = [-0.04413108 -0.2074192   0.08954874  0.4259652 ], reward = 1.0, done = False
n_state = [-0.04827946 -0.40368786  0.09806804  0.7454807 ], reward = 1.0, done = False
n_state = [-0.05635322 -0.2100461   0.11297766  0.4851995 ], reward = 1.0, done = False
n_state = [-0.06055414 -0.4065659   0.12268165  0.8112448 ], reward = 1.0, done = False
n_state = [-0.06868546 -0.21331897  0.13890655  0.55953103], reward = 1.0, done = False
n_state = [-0.07295184 -0.02039196  0.15009716  0.31363523], reward = 1.0, done = False
n_state = [-0.07335968 -0.21729782  0.15636986  0.64963627], reward = 1.0, done = False
n_state = [-0.07770564 -0.414212

# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [7]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [8]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 1.1974113e+00, -1.4069810e+38, -3.6003578e-01, -2.0436207e+38],
      dtype=float32)

# 3. Train an RL Model

In [9]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [10]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 1937 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1197        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009211546 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.0125      |
|    learning_rate        | 0.0003      |
|    loss                 | 7.71        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 58.3        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [10]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [14]:
print(PPO_path)

Training\Saved Models\PPO_model


In [15]:
model.save(PPO_path)

D:\Anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [16]:
del model

In [11]:
model = PPO.load(PPO_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# 4. Evaluation

In [12]:
from stable_baselines3.common.evaluation import evaluate_policy

In [13]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

D:\Anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [21]:
env.close()

# 5. Test Model

In [4]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

NameError: name 'env' is not defined

In [1]:
env.close()

NameError: name 'env' is not defined

# 6. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()